In [ ]:
import pandas as pd
import numpy as np
# Load the data
df = pd.read_csv("online_retail_II.csv")
# drop duplicates
df = df.drop_duplicates()
# drop missing customer ID
df = df.dropna(subset=['Customer ID'])
# Remove canceled invoice
df = df[~df['Invoice'].astype(str).str.startswith('C')]
# Convert Date column
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# Inserting a Total Column
df['Total Amount'] = df['Quantity'] * df['Price']
df.head()

In [ ]:
# Save Cleaned File
df.to_csv("Cleaned_retail_data.csv", index=False)


In [ ]:

import pandas as pd
from sqlalchemy import create_engine

# === CONFIGURATION ===
csv_file_path = r'C:\Users\Lenovo\Desktop\My Folder\Excel\Online Retail\cleaned_retail_data.csv'    # <-- Change this
mysql_user = 'root'            # <-- Change this
mysql_password = '12345'              # <-- Change this
mysql_host = 'localhost'                      # or IP address
mysql_port = 3306                             # Default MySQL port
mysql_db = 'retail'               # <-- Change this
mysql_table = 'cleaned_retail_data'               # <-- Change this

chunk_size = 50000  # Adjust depending on your system’s memory

# === SQLAlchemy Connection String ===
engine_str = f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_db}"
engine = create_engine(engine_str)

# === Start Importing in Chunks ===
print("🚀 Starting CSV import...")

try:
    for i, chunk in enumerate(pd.read_csv(csv_file_path, chunksize=chunk_size)):
        print(f"📦 Inserting chunk {i+1}...")
        chunk.to_sql(name=mysql_table, con=engine, if_exists='append', index=False)
        print(f"✅ Chunk {i+1} inserted successfully.")
except Exception as e:
    print(f"❌ Error during import: {e}")

print("🎉 CSV import completed!")


In [21]:
# RFM Customer Segmentation
from datetime import datetime
ref_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# group by customer
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate' : lambda x: (ref_date - x.max()).days,
    'Invoice' : 'nunique',
    'Total Amount' : 'sum'
})
rfm.rename(columns={
    'InvoiceDate' : 'Recency',
    'Invoice' : 'Frequency',
    'Total Amount' : 'Monetary'
}, inplace=True)


In [22]:
rfm

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,326,12,77556.46
12347.0,2,8,4921.53
12348.0,75,5,2019.40
12349.0,19,4,4428.69
12350.0,310,1,334.40
...,...,...,...
18283.0,4,22,2664.90
18284.0,432,1,461.68
18285.0,661,1,427.00


In [25]:
# Create RFM scores
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])
rfm['RFM_score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)


In [26]:
rfm

,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_score
Customer ID,,,,,,,
12346.0,326,12,77556.46,2,5,5,255
12347.0,2,8,4921.53,5,4,5,545
12348.0,75,5,2019.40,3,4,4,344
12349.0,19,4,4428.69,5,3,5,535
12350.0,310,1,334.40,2,1,2,212
...,...,...,...,...,...,...,...
18283.0,4,22,2664.90,5,5,4,554
18284.0,432,1,461.68,1,2,2,122
18285.0,661,1,427.00,1,2,2,122
